In [34]:
import tensorflow as tf 

In [35]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y.%m.%d-%H.%M.%S")
root_logdir = "tensor_logs"
logdir = "/Users/ash/Downloads/tensorflow/{}/run-{}/".format(root_logdir, now)

In [36]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets("/Users/ash/Downloads/tensorflow/data/")

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /Users/ash/Downloads/tensorflow/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /Users/ash/Downloads/tensorflow/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /Users/ash/Downloads/tensorflow/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /Users/ash/Downloads/tensorflow/data/t10k-labels-idx1-ubyte.gz


In [40]:
inputs= 28*28  
hidden1 = 512
hidden2 = 300
outputs = 10
alpha=0.01

X=tf.placeholder(tf.float32, shape=(None,inputs), name='X')
y=tf.placeholder(tf.int32, shape=(None), name='y')

def layer(X, neurons, name, activ):
    with tf.name_scope(name):
        inputs = int(X.get_shape()[1])
        std = 2 / np.sqrt(inputs)
        W = tf.Variable(tf.truncated_normal((inputs, neurons), stddev=std), name="weights")
        b = tf.Variable(tf.zeros([neurons]), name="biases")
        z = tf.matmul(X, W) + b
        if activ=="relu":
            return tf.nn.relu(z)
        else:
            return z
        
with tf.name_scope('DenseNet'): 
    layer1=layer(X, neurons=hidden1, name='hidden1', activ='relu')
    layer2=layer(X, neurons=hidden2, name='hidden2', activ='relu')
    meta_output=layer(X, neurons=outputs, name='meta_out', activ=None)

with tf.name_scope("loss"):
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
                   labels=y, logits=meta_output)
    loss = tf.reduce_mean(entropy, name="loss") 
    
with tf.name_scope("GDOptimizer"):
    optimizer = tf.train.GradientDescentOptimizer(alpha)
    training_op = optimizer.minimize(loss)

with tf.name_scope("softmax"):
    correct = tf.nn.in_top_k(meta_output, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver() 

In [47]:
train_summary = tf.summary.scalar('TrainAccuracy', acc_train)
test_summary = tf.summary.scalar('TestAccuracy', acc_test)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [51]:
batch=50
epoch=20

with tf.Session() as sess:
    init.run()
    for epoch in range(epoch):
        for iteration in range(data.train.num_examples // batch):
            X_batch, y_batch = data.train.next_batch(batch)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: data.test.images, y: data.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    save_path = saver.save(sess, "/Users/ash/Downloads/tensorflow/densenn.ckpt") 

0 Train accuracy: 0.78 Test accuracy: 0.8592
1 Train accuracy: 0.96 Test accuracy: 0.8825
2 Train accuracy: 0.88 Test accuracy: 0.893
3 Train accuracy: 0.86 Test accuracy: 0.8988
4 Train accuracy: 0.94 Test accuracy: 0.9008
5 Train accuracy: 0.92 Test accuracy: 0.9048
6 Train accuracy: 0.9 Test accuracy: 0.9071
7 Train accuracy: 0.88 Test accuracy: 0.9094
8 Train accuracy: 0.98 Test accuracy: 0.91
9 Train accuracy: 0.88 Test accuracy: 0.9111
10 Train accuracy: 0.9 Test accuracy: 0.9115
11 Train accuracy: 0.88 Test accuracy: 0.9128
12 Train accuracy: 0.98 Test accuracy: 0.9144
13 Train accuracy: 0.94 Test accuracy: 0.9147
14 Train accuracy: 0.92 Test accuracy: 0.9156
15 Train accuracy: 0.94 Test accuracy: 0.9146
16 Train accuracy: 0.86 Test accuracy: 0.9166
17 Train accuracy: 0.96 Test accuracy: 0.9159
18 Train accuracy: 0.92 Test accuracy: 0.916
19 Train accuracy: 0.88 Test accuracy: 0.9162


In [50]:
file_writer.close()